## Initialize the `SparkSession`

In [1]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('final_project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=final_project-sciuto>

In [2]:
spark = SparkSession(sc)

In [3]:
from datetime import datetime

import pyspark.sql.functions as functions

### load whole dataset

In [4]:
whole_df = spark.read.csv("/datasets/project/istdaten/*/*/*.csv", header=True, sep=";")

In [5]:
whole_df.count()

196232995

- rename some useful columns

In [5]:
oldColumns = whole_df.schema.names
print(oldColumns)
newColumns = ["date", 'trip_id', 
              'BETREIBER_ID', 'BETREIBER_ABK',
              'BETREIBER_NAME', "transport_type", 
             "train_line", "train_service", 
              'UMLAUF_ID', 'VERKEHRSMITTEL_TEXT',
             "additional_trip", "failed_trip",
             'BPUIC', "station_name", "arrival_time",
             "actual_arrival", 'AN_PROGNOSE_STATUS',
             "departure_time", "actual_departure",
             'AB_PROGNOSE_STATUS', "DURCHFAHRT_TF"]

['BETRIEBSTAG', 'FAHRT_BEZEICHNER', 'BETREIBER_ID', 'BETREIBER_ABK', 'BETREIBER_NAME', 'PRODUKT_ID', 'LINIEN_ID', 'LINIEN_TEXT', 'UMLAUF_ID', 'VERKEHRSMITTEL_TEXT', 'ZUSATZFAHRT_TF', 'FAELLT_AUS_TF', 'BPUIC', 'HALTESTELLEN_NAME', 'ANKUNFTSZEIT', 'AN_PROGNOSE', 'AN_PROGNOSE_STATUS', 'ABFAHRTSZEIT', 'AB_PROGNOSE', 'AB_PROGNOSE_STATUS', 'DURCHFAHRT_TF']


In [6]:
whole_df = whole_df.toDF(*newColumns)
whole_df.printSchema()
# whole_df.show()

root
 |-- date: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- BETREIBER_ID: string (nullable = true)
 |-- BETREIBER_ABK: string (nullable = true)
 |-- BETREIBER_NAME: string (nullable = true)
 |-- transport_type: string (nullable = true)
 |-- train_line: string (nullable = true)
 |-- train_service: string (nullable = true)
 |-- UMLAUF_ID: string (nullable = true)
 |-- VERKEHRSMITTEL_TEXT: string (nullable = true)
 |-- additional_trip: string (nullable = true)
 |-- failed_trip: string (nullable = true)
 |-- BPUIC: string (nullable = true)
 |-- station_name: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- actual_arrival: string (nullable = true)
 |-- AN_PROGNOSE_STATUS: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- actual_departure: string (nullable = true)
 |-- AB_PROGNOSE_STATUS: string (nullable = true)
 |-- DURCHFAHRT_TF: string (nullable = true)



- dropping useless columns:

In [8]:
whole_df = whole_df.drop('BETREIBER_ID','BETREIBER_ABK', 'BETREIBER_NAME', 'UMLAUF_ID', 'BPUIC')

In [9]:
whole_df.select('transport_type').distinct().show()

+--------------+
|transport_type|
+--------------+
|           BUS|
|          null|
|          Tram|
|           Zug|
|           Bus|
|        Schiff|
+--------------+



- loading stations csv in the 10 Km radius:

In [9]:
radius_stations_df = spark.read.csv('final_project/zurich_hb_stops.csv', header=True, sep=",")

In [10]:
radius_stations_df.count()

1039

- Inner join on station name in order to work on the stations in the 10 Km radius:

In [11]:
filtered_df = whole_df.join(radius_stations_df, on="station_name", how='inner')

In [12]:
filtered_df.select('station_name').distinct().count()

971

In [14]:
filtered_df.select('transport_type').distinct().show()

+--------------+
|transport_type|
+--------------+
|           BUS|
|          null|
|          Tram|
|           Zug|
|           Bus|
+--------------+



- First assumption for the stop time-table: we assume that the time table is the same for each regular day, we choose this day because in homework 2 we verified it was a regular day, also we make sure that the public transport stops at the specific station using the flag DURCHFAHRT_TF:

In [13]:
first_table = filtered_df.filter((filtered_df.date == "17.10.2017")\
                                 & (filtered_df.DURCHFAHRT_TF == False) \
                                 & (filtered_df.additional_trip == False)).cache()

In [16]:
first_table.count()

233516

- removing rows that do not have a next stop in the radius:

In [14]:
useful_trips = first_table.groupBy('trip_id').count().filter("count > 1")

In [15]:
useful_trips.count()

16282

In [16]:
useful_trips = useful_trips.drop('count')

- merging with our first table:

In [17]:
first_table = first_table.join(useful_trips, on="trip_id", how='inner').cache()

In [21]:
first_table.count()

233087

- We successfully removed nearly 500 trips that have no next stop in our radius

In [19]:
from pyspark.sql.functions import *

- insert some valid values for arrival and departure time when they are null, in order to not have any problem during the orderBy dates.

In [23]:
first_table_final = first_table\
.withColumn('departure_New', when(first_table.departure_time.isNull(), 
                                 first_table.arrival_time).otherwise(first_table.departure_time))\
.withColumn('arrival_New', when(first_table.arrival_time.isNull(), 
                                 first_table.departure_time).otherwise(first_table.arrival_time))\
.drop('arrival_time', 'departure_time')\
.select(col('station_name'), 
        col('trip_id'),
        col('departure_New').substr(12, 5).alias('departure_time'), 
        col('arrival_New').substr(12, 5).alias('arrival_time'), 
        col('train_line'),
        col('transport_type')
        )\


In [24]:
first_table_final.show(5)

+----------------+--------------+--------------+------------+----------+--------------+
|    station_name|       trip_id|departure_time|arrival_time|train_line|transport_type|
+----------------+--------------+--------------+------------+----------+--------------+
|       Zürich HB|85:11:1507:002|         06:39|       06:30|      1507|           Zug|
|Zürich Flughafen|85:11:1507:002|         06:51|       06:49|      1507|           Zug|
|       Zürich HB|85:11:1509:002|         07:39|       07:30|      1509|           Zug|
|Zürich Flughafen|85:11:1509:002|         07:51|       07:49|      1509|           Zug|
|Zürich Flughafen|85:11:1510:002|         07:13|       07:11|      1510|           Zug|
+----------------+--------------+--------------+------------+----------+--------------+
only showing top 5 rows



- Ordering our time-tables:

In [25]:
first_table_final = first_table_final.orderBy(desc('station_name'), 'arrival_time', 'departure_time').dropDuplicates()

In [25]:
first_table = first_table.orderBy('trip_id', 'train_line', 'departure_time', 'arrival_time').dropDuplicates()

In [26]:
first_table_final.show(5)

+--------------------+--------------------+--------------+------------+-----------+--------------+
|        station_name|             trip_id|departure_time|arrival_time| train_line|transport_type|
+--------------------+--------------------+--------------+------------+-----------+--------------+
|Zürich,Kalkbreite...|85:3849:80541-020...|         00:03|       00:02|85:3849:002|          Tram|
|Zürich,Kalkbreite...|85:3849:80396-020...|         00:07|       00:06|85:3849:002|          Tram|
|Zürich,Kalkbreite...|85:849:85422-02032-1|         00:07|       00:07| 85:849:032|           Bus|
|Zürich,Kalkbreite...|85:3849:80729-020...|         00:13|       00:12|85:3849:003|          Tram|
|Zürich,Kalkbreite...|85:3849:80886-020...|         00:13|       00:12|85:3849:003|          Tram|
+--------------------+--------------------+--------------+------------+-----------+--------------+
only showing top 5 rows



In [27]:
first_table_final.count()

233067

There were 6 thousands duplicates

In [28]:
first_table_final = first_table_final.dropna()

In [29]:
first_table_final.count()

233067

no presence of null values

In [30]:
first_table_final.select('transport_type').distinct().show()

+--------------+
|transport_type|
+--------------+
|          Tram|
|           Zug|
|           Bus|
+--------------+



- showing the number of distinct stops:

In [ ]:
first_table_final.select('station_name').distinct().count()

- showing the number of distinct lines:


In [31]:
first_table_final.filter((first_table_final.transport_type == 'Zug')).select('train_line').distinct().count()

1866

In [32]:
first_table_final.filter(first_table_final.transport_type == 'Bus').select('train_line').distinct().count()

90

In [33]:
first_table_final.filter(first_table_final.transport_type == 'Tram').select('train_line').distinct().count()

15

- saving locally our first table:

In [26]:
pd = first_table.toPandas()

In [27]:
pd.loc[pd.trip_id == '85:882:6610-36101-1', ['trip_id', 'station_name', 'departure_time', 'arrival_time', 'train_line']].head()

,trip_id,station_name,departure_time,arrival_time,train_line
232924,85:882:6610-36101-1,"Bassersdorf, Bahnhof",17.10.2017 12:16,None,85:882:660
232925,85:882:6610-36101-1,"Bassersdorf, Dietlikonerstr.",17.10.2017 12:17,17.10.2017 12:16,85:882:660
232926,85:882:6610-36101-1,"Bassersdorf, Gemeindehaus",17.10.2017 12:18,17.10.2017 12:17,85:882:660


In [34]:
pd = first_table_final.toPandas()

In [35]:
pd.head(20)

,station_name,trip_id,departure_time,arrival_time,train_line,transport_type
0,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80541-02002-1,00:03,00:02,85:3849:002,Tram
1,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80396-02002-1,00:07,00:06,85:3849:002,Tram
2,"Zürich,Kalkbreite/Bhf.Wiedikon",85:849:85422-02032-1,00:07,00:07,85:849:032,Bus
3,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80729-02003-1,00:13,00:12,85:3849:003,Tram
4,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80886-02003-1,00:13,00:12,85:3849:003,Tram
5,"Zürich,Kalkbreite/Bhf.Wiedikon",85:849:85529-02032-1,00:14,00:14,85:849:032,Bus
6,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80335-02002-1,00:18,00:17,85:3849:002,Tram
7,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80462-02002-1,00:22,00:21,85:3849:002,Tram
8,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80824-02003-1,00:27,00:27,85:3849:003,Tram
9,"Zürich,Kalkbreite/Bhf.Wiedikon",85:3849:80602-02003-1,00:28,00:27,85:3849:003,Tram


In [36]:
pd.shape

(233067, 6)

In [ ]:
pd.to_csv('pandas_station_time_tables.csv')

### Second table
- now we have to create a table where we have the time-table of a regular day for all the lines that we have found before (2297 train_lines, 90 bus lines, 15 tram lines).

## using trip id

In [37]:
second_table = first_table_final

- taking only the trips that have a next stop in our radius:

In [38]:
useful_transport_df = second_table.groupBy('trip_id').count().filter("count > 1")

In [39]:
useful_transport_df.orderBy('trip_id').show(20)

+--------------+-----+
|       trip_id|count|
+--------------+-----+
|85:11:1507:002|    2|
|85:11:1509:002|    2|
|85:11:1510:002|    2|
|85:11:1511:002|    2|
|85:11:1512:002|    2|
|85:11:1513:003|    2|
|85:11:1514:002|    2|
|85:11:1515:003|    2|
|85:11:1516:002|    2|
|85:11:1517:003|    2|
|85:11:1518:002|    2|
|85:11:1519:003|    2|
|85:11:1520:002|    2|
|85:11:1521:003|    2|
|85:11:1522:002|    2|
|85:11:1523:003|    2|
|85:11:1524:002|    2|
|85:11:1525:003|    2|
|85:11:1526:002|    2|
|85:11:1527:003|    2|
+--------------+-----+
only showing top 20 rows



In [40]:
useful_transport_df = useful_transport_df.drop('count')

- merging with our main second table:

In [41]:
second_table_useful = second_table.join(useful_transport_df, on="trip_id", how='inner')

In [42]:
second_table_useful.select('train_line').distinct().count()

1971

In [43]:
second_table_useful.select('trip_id').distinct().count()

16282

- this result means that there are public transports that do more trips in the same day.

In [44]:
second_table_useful = second_table_useful.orderBy('trip_id',
                                                  'arrival_time', 
                                                  'departure_time', 
                                                  'train_line', 
                                                  'station_name'
                                                 )

In [45]:
second_table_useful.show(50)

+--------------+----------------+--------------+------------+----------+--------------+
|       trip_id|    station_name|departure_time|arrival_time|train_line|transport_type|
+--------------+----------------+--------------+------------+----------+--------------+
|85:11:1507:002|       Zürich HB|         06:39|       06:30|      1507|           Zug|
|85:11:1507:002|Zürich Flughafen|         06:51|       06:49|      1507|           Zug|
|85:11:1509:002|       Zürich HB|         07:39|       07:30|      1509|           Zug|
|85:11:1509:002|Zürich Flughafen|         07:51|       07:49|      1509|           Zug|
|85:11:1510:002|Zürich Flughafen|         07:13|       07:11|      1510|           Zug|
|85:11:1510:002|       Zürich HB|         07:30|       07:23|      1510|           Zug|
|85:11:1511:002|       Zürich HB|         08:39|       08:30|      1511|           Zug|
|85:11:1511:002|Zürich Flughafen|         08:51|       08:49|      1511|           Zug|
|85:11:1512:002|Zürich Flughafen

- now we will use a window on each trip item to obtain our desired table:

In [46]:
from pyspark.sql.window import Window

In [47]:
trip_window = Window.partitionBy("trip_id").orderBy(unix_timestamp('arrival_time','HH:mm'),
                                                    unix_timestamp('departure_time','HH:mm'))

- defining the windows function in order to use the next row in the partition. We create two new columns modeling the next stop and the arrival time in the next stop:

In [48]:
next_station = lag("station_name", -1).over(trip_window)
next_station_arrival = lag("arrival_time", -1).over(trip_window)

In [49]:
second_table_useful_final = second_table_useful\
                                               .withColumn('nx_station', next_station)\
                                               .withColumn('arr_nx', next_station_arrival)

In [50]:
second_table_useful_final = second_table_useful_final.drop('arrival_time')

- reordering the columns:

In [51]:
second_table_useful_final = second_table_useful_final.select('trip_id',
                                                             'transport_type',
                                                             'train_line',
                                                             'station_name',
                                                             'departure_time',
                                                             'nx_station',
                                                             'arr_nx')

In [52]:
second_table_useful_final.show(200)

+--------------------+--------------+-----------+--------------------+--------------+--------------------+------+
|             trip_id|transport_type| train_line|        station_name|departure_time|          nx_station|arr_nx|
+--------------------+--------------+-----------+--------------------+--------------+--------------------+------+
|     85:11:18388:001|           Zug|      18388|           Dietlikon|         23:13|           Stettbach| 23:17|
|     85:11:18388:001|           Zug|      18388|           Stettbach|         23:18|  Zürich Stadelhofen| 23:22|
|     85:11:18388:001|           Zug|      18388|  Zürich Stadelhofen|         23:23|           Zürich HB| 23:26|
|     85:11:18388:001|           Zug|      18388|           Zürich HB|         23:29|   Zürich Hardbrücke| 23:31|
|     85:11:18388:001|           Zug|      18388|   Zürich Hardbrücke|         23:31|   Zürich Altstetten| 23:35|
|     85:11:18388:001|           Zug|      18388|   Zürich Altstetten|         23:36|   

In [53]:
second_table_useful_final.count()

233067

- showing trains:

In [54]:
second_table_useful_final\
.filter(second_table_useful_final.transport_type == "Zug")\
.orderBy('train_line', 'trip_id', 'departure_time', 'arr_nx').show(50)

+-------------+--------------+----------+--------------------+--------------+--------------------+------+
|      trip_id|transport_type|train_line|        station_name|departure_time|          nx_station|arr_nx|
+-------------+--------------+----------+--------------------+--------------+--------------------+------+
|85:46:142:000|           Zug|       142|          Neue Forch|         05:16|            Maiacher| 05:17|
|85:46:142:000|           Zug|       142|            Maiacher|         05:17|             Zumikon| 05:18|
|85:46:142:000|           Zug|       142|             Zumikon|         05:18|            Waltikon| 05:20|
|85:46:142:000|           Zug|       142|            Waltikon|         05:20|        Zollikerberg| 05:22|
|85:46:142:000|           Zug|       142|        Zollikerberg|         05:22| Spital Zollikerberg| 05:23|
|85:46:142:000|           Zug|       142| Spital Zollikerberg|         05:23|            Waldburg| 05:24|
|85:46:142:000|           Zug|       142|     

- showing buses:

In [55]:
second_table_useful_final\
.filter(second_table_useful_final.transport_type == "Bus")\
.orderBy('train_line', 'trip_id', 'departure_time', 'arr_nx').show(50)

+--------------------+--------------+----------+--------------------+--------------+--------------------+------+
|             trip_id|transport_type|train_line|        station_name|departure_time|          nx_station|arr_nx|
+--------------------+--------------+----------+--------------------+--------------+--------------------+------+
|85:773:22519-01485-1|           Bus|85:773:454|Regensdorf-Watt, ...|         06:27| Regensdorf, Althard| 06:27|
|85:773:22519-01485-1|           Bus|85:773:454| Regensdorf, Althard|         06:28|Regensdorf, Hardh...| 06:28|
|85:773:22519-01485-1|           Bus|85:773:454|Regensdorf, Hardh...|         06:28|Regensdorf, Quers...| 06:29|
|85:773:22519-01485-1|           Bus|85:773:454|Regensdorf, Quers...|         06:29|                null|  null|
|85:773:98965-01485-1|           Bus|85:773:454| Regensdorf, Althard|         07:02|Regensdorf-Watt, ...| 07:02|
|85:773:98965-01485-1|           Bus|85:773:454|Regensdorf-Watt, ...|         07:02|Regensdorf, 

- showing trams:

In [56]:
second_table_useful_final\
.filter(second_table_useful_final.transport_type == "Tram")\
.orderBy('train_line', 'trip_id', 'departure_time', 'arr_nx').show(50)

+--------------------+--------------+-----------+--------------------+--------------+--------------------+------+
|             trip_id|transport_type| train_line|        station_name|departure_time|          nx_station|arr_nx|
+--------------------+--------------+-----------+--------------------+--------------+--------------------+------+
|85:3849:80250-020...|          Tram|85:3849:002|Zürich,Kalkbreite...|         06:01|   Zürich, Lochergut| 06:02|
|85:3849:80250-020...|          Tram|85:3849:002|   Zürich, Lochergut|         06:02|Zürich, Zypressen...| 06:03|
|85:3849:80250-020...|          Tram|85:3849:002|Zürich, Zypressen...|         06:03|Zürich, Albisried...| 06:04|
|85:3849:80250-020...|          Tram|85:3849:002|Zürich, Albisried...|         06:04|  Zürich, Letzigrund| 06:05|
|85:3849:80250-020...|          Tram|85:3849:002|  Zürich, Letzigrund|         06:06|Zürich, Freihofst...| 06:06|
|85:3849:80250-020...|          Tram|85:3849:002|Zürich, Freihofst...|         06:07|   

- Saving our second table in pandas:

In [57]:
second_table_useful_final_pandas = second_table_useful_final.toPandas()

In [ ]:
second_table_useful_final_pandas.to_csv('trips_time_tables_pandas.csv')

In [58]:
second_pd = pd.from_csv('trips_time_tables_pandas.csv')

In [59]:
second_pd.head(20)

,trip_id,transport_type,train_line,station_name,departure_time,nx_station,arr_nx
0,85:11:18388:001,Zug,18388,Dietlikon,23:13,Stettbach,23:17
1,85:11:18388:001,Zug,18388,Stettbach,23:18,Zürich Stadelhofen,23:22
2,85:11:18388:001,Zug,18388,Zürich Stadelhofen,23:23,Zürich HB,23:26
3,85:11:18388:001,Zug,18388,Zürich HB,23:29,Zürich Hardbrücke,23:31
4,85:11:18388:001,Zug,18388,Zürich Hardbrücke,23:31,Zürich Altstetten,23:35
5,85:11:18388:001,Zug,18388,Zürich Altstetten,23:36,Schlieren,23:38
6,85:11:18388:001,Zug,18388,Schlieren,23:38,Glanzenberg,23:40
7,85:11:18388:001,Zug,18388,Glanzenberg,23:40,NaN,NaN
8,85:11:18718:001,Zug,18718,Zürich Tiefenbrunnen,05:41,Zürich Stadelhofen,05:44
9,85:11:18718:001,Zug,18718,Zürich Stadelhofen,05:45,Zürich HB,05:47


In [60]:
second_pd.shape

(233067, 7)

### THIRD TABLE: COMPUTING THE DELTAS DELAYS

- First assumptions: we filter entries were transport does not stop, entries that are additional trips and failed trips

In [61]:
third_table = filtered_df.filter((filtered_df.DURCHFAHRT_TF == False) \
                                 & (filtered_df.additional_trip == False)\
                                 & (filtered_df.failed_trip == False)).cache()

In [62]:
third_table.count()

51046583

In [63]:
third_table.select('trip_id').distinct().count()

153603

In [64]:
useful_trips = third_table.groupBy('trip_id').count().filter("count > 1")

In [65]:
useful_trips.count()

153572

In [66]:
useful_trips = useful_trips.drop('count')

- merging with our third table:

In [67]:
third_table = third_table.join(useful_trips, on="trip_id", how='inner').cache()

In [68]:
third_table.count()

51046552

In [69]:
third_table.select(third_table.AN_PROGNOSE_STATUS).distinct().show()

+------------------+
|AN_PROGNOSE_STATUS|
+------------------+
|        GESCHAETZT|
|          PROGNOSE|
|              REAL|
|         UNBEKANNT|
+------------------+



In [70]:
third_table.select(third_table.AB_PROGNOSE_STATUS).distinct().show()

+------------------+
|AB_PROGNOSE_STATUS|
+------------------+
|        GESCHAETZT|
|          PROGNOSE|
|              REAL|
|         UNBEKANNT|
+------------------+



In [71]:
third_table.filter(third_table.AN_PROGNOSE_STATUS == "REAL").count()

10312

In [72]:
third_table.filter(third_table.AB_PROGNOSE_STATUS == "REAL").count()

10443

- We have 10.312 entries with REAL = effective actual time of arrival, 10.443 for departure

In [73]:
third_table.filter(third_table.AN_PROGNOSE_STATUS == "GESCHAETZT").count()

2479631

In [74]:
third_table.filter(third_table.AB_PROGNOSE_STATUS == "GESCHAETZT").count()

2505089

- we have 2.479.647 entries with GESCHAETZT = calculated actual time of arrival, 2.505.094 for departure

In [75]:
third_table.filter(third_table.AN_PROGNOSE_STATUS == "PROGNOSE").count()

48484828

In [76]:
third_table.filter(third_table.AB_PROGNOSE_STATUS == "PROGNOSE").count()

48459245

- we have 48.484.841 entries with PROGNOSE = arrival forecast, 48.484.841 also for departure


In [77]:
third_table.filter(third_table.AN_PROGNOSE_STATUS == "UNBEKANNT").count()

71781

In [78]:
third_table.filter(third_table.AB_PROGNOSE_STATUS == "UNBEKANNT").count()

71775

- we have 71.783 entries with UNBEKANNT = Unknown, No forecast and actual times available for this and all previous stops, 71.775 for departures

In [79]:
third_table.filter(third_table.AN_PROGNOSE_STATUS == "UNBEKANNT").select('actual_arrival').show(5)

+--------------+
|actual_arrival|
+--------------+
|          null|
|          null|
|          null|
|          null|
|          null|
+--------------+
only showing top 5 rows



- We count the null values of actual arrivals and actual departures:

In [80]:
third_table.filter(third_table.actual_arrival.isNull()).count()

2666554

In [81]:
third_table.filter(third_table.actual_departure.isNull()).count()

2669517

- we have more values than in the count() on the UNBEKANNT, this means that we have null values also in the other categories --> let us check:

In [82]:
third_table.filter(third_table.actual_arrival.isNull()).groupBy('AN_PROGNOSE_STATUS').count().show()

+------------------+-------+
|AN_PROGNOSE_STATUS|  count|
+------------------+-------+
|          PROGNOSE|2594773|
|         UNBEKANNT|  71781|
+------------------+-------+



In [83]:
third_table.filter(third_table.actual_departure.isNull()).groupBy('AB_PROGNOSE_STATUS').count().show()

+------------------+-------+
|AB_PROGNOSE_STATUS|  count|
+------------------+-------+
|          PROGNOSE|2597742|
|         UNBEKANNT|  71775|
+------------------+-------+



- As you can see, all the UNBEKANNT entries have actual_arrival = null, and some entries with PROGNOSE have also null values. For the null values in PROGNOSE we suspect that maybe this could be related to the entries that represent the first or the last stop, where respectively the arrival and departure times are null. We will verify this hypothesis:

In [84]:
third_table.filter((third_table.actual_departure.isNull()) \
                   & (third_table.departure_time.isNull()))\
                   .groupBy('AB_PROGNOSE_STATUS').count().show()

+------------------+-------+
|AB_PROGNOSE_STATUS|  count|
+------------------+-------+
|          PROGNOSE|2566167|
+------------------+-------+



In [85]:
third_table.filter((third_table.actual_departure.isNull()) \
                   & (third_table.departure_time.isNotNull()))\
                   .groupBy('AB_PROGNOSE_STATUS').count().show()

+------------------+-----+
|AB_PROGNOSE_STATUS|count|
+------------------+-----+
|          PROGNOSE|31575|
|         UNBEKANNT|71775|
+------------------+-----+



In [86]:
third_table.filter((third_table.actual_arrival.isNull()) \
                   & (third_table.arrival_time.isNull()))\
                   .groupBy('AN_PROGNOSE_STATUS').count().show()

+------------------+-------+
|AN_PROGNOSE_STATUS|  count|
+------------------+-------+
|          PROGNOSE|2565026|
+------------------+-------+



In [87]:
third_table.filter((third_table.actual_arrival.isNull()) \
                   & (third_table.arrival_time.isNotNull()))\
                   .groupBy('AN_PROGNOSE_STATUS').count().show()

+------------------+-----+
|AN_PROGNOSE_STATUS|count|
+------------------+-----+
|          PROGNOSE|29747|
|         UNBEKANNT|71781|
+------------------+-----+



- For departure times, we have that 2.566.183 null values correspond to entries representing a last stop, we will treat these delays also as null. Then we have 31.575 null values for the departure actual time when the departure time is different from null, we will treat these delays as zeros.
- Same procedure for arrivals

- We will see how many first and last stop are linked to the transport type:

In [88]:
third_table.filter(third_table.arrival_time.isNull()).groupBy('transport_type').count().show()

+--------------+-------+
|transport_type|  count|
+--------------+-------+
|          null|     24|
|          Tram| 734985|
|           Zug| 149176|
|           Bus|1680841|
+--------------+-------+



In [89]:
third_table.filter(third_table.departure_time.isNull()).groupBy('transport_type').count().show()

+--------------+-------+
|transport_type|  count|
+--------------+-------+
|          null|      4|
|          Tram| 733309|
|           Zug| 150648|
|           Bus|1682206|
+--------------+-------+



- if we just use only the GESCHAETZT entries, we will use only 4.8 % of our available data. Also, if we drop all the other entries there is the risk to lose information about some connection in our network. Right now we will just compute the difference of the columns. A further discussion with the other group members and TAs should be done.
- Is critical how to handle the null values, do we put delay = 0 if null? Or do we drop those entries? (by dropping them, we stil have the same problem highlighted in the first point of this cell. Right now I put a zero for all the null values.
- We also create a 'hour' column because it could be useful for the statitistics tests

In [90]:
third_table_final = third_table\
.withColumn('arrival_delay', when((third_table.actual_arrival.isNull()) \
                                  & (third_table.arrival_time.isNull()), None
                                 )
                                 .when((third_table.actual_arrival.isNull()) \
                                       & (third_table.arrival_time.isNotNull()), 0)\
                                 .otherwise(functions.round(unix_timestamp("actual_arrival",'dd.MM.yyyy HH:mm') - \
                                            unix_timestamp("arrival_time",'dd.MM.yyyy HH:mm')) / 60))\

.withColumn('departure_delay', when((third_table.actual_departure.isNull())\
                                    & (third_table.departure_time.isNull()), None
                                 )
                                 .when((third_table.actual_departure.isNull()) \
                                       & (third_table.departure_time.isNotNull()), 0)\
                                    .otherwise(functions.round(unix_timestamp("actual_departure",'dd.MM.yyyy HH:mm')\
                                               - unix_timestamp("departure_time",'dd.MM.yyyy HH:mm')) /60))\
.withColumn('hour',  when(third_table.arrival_time.isNull(), hour(to_timestamp(third_table.departure_time,
                                                                              'dd.MM.yyyy HH:mm'))) \
                         .otherwise(hour(to_timestamp(third_table.arrival_time, 'dd.MM.yyyy HH:mm'))))

In [91]:
third_table_final.select('arrival_time', 
                   'actual_arrival', 
                   'arrival_delay', 
                   'departure_time', 
                   'actual_departure', 
                   'departure_delay',
                   'hour').show(5)

+----------------+-------------------+-------------+----------------+-------------------+---------------+----+
|    arrival_time|     actual_arrival|arrival_delay|  departure_time|   actual_departure|departure_delay|hour|
+----------------+-------------------+-------------+----------------+-------------------+---------------+----+
|09.12.2017 01:52|09.12.2017 01:52:27|          0.0|09.12.2017 01:52|09.12.2017 01:53:53|            1.0|   1|
|09.12.2017 01:55|09.12.2017 01:55:33|          0.0|09.12.2017 01:57|09.12.2017 01:57:49|            0.0|   1|
|09.12.2017 01:59|09.12.2017 01:59:39|          0.0|09.12.2017 01:59|09.12.2017 02:02:46|            3.0|   1|
|09.12.2017 02:01|09.12.2017 02:05:28|          4.0|09.12.2017 02:01|09.12.2017 02:06:19|            5.0|   2|
|09.12.2017 02:06|09.12.2017 02:10:08|          4.0|09.12.2017 02:06|09.12.2017 02:11:02|            5.0|   2|
+----------------+-------------------+-------------+----------------+-------------------+---------------+----+
o

In [92]:
third_table_final.select('hour').distinct().orderBy('hour').show(25)

+----+
|hour|
+----+
|   0|
|   1|
|   2|
|   3|
|   4|
|   5|
|   6|
|   7|
|   8|
|   9|
|  10|
|  11|
|  12|
|  13|
|  14|
|  15|
|  16|
|  17|
|  18|
|  19|
|  20|
|  21|
|  22|
|  23|
+----+



- Here we just check if our new columns were created properly with the "when" case:

In [93]:
third_table_final.filter((third_table_final.actual_arrival.isNull()) \
                   & (third_table_final.arrival_time.isNotNull()))\
                   .select('arrival_time', 
                           'actual_arrival', 
                           'arrival_delay', 
                           'departure_time', 
                           'actual_departure', 
                           'departure_delay',
                           'hour').show(5)

+----------------+--------------+-------------+----------------+----------------+---------------+----+
|    arrival_time|actual_arrival|arrival_delay|  departure_time|actual_departure|departure_delay|hour|
+----------------+--------------+-------------+----------------+----------------+---------------+----+
|15.04.2018 23:31|          null|          0.0|15.04.2018 23:31|            null|            0.0|  23|
|15.04.2018 23:35|          null|          0.0|15.04.2018 23:36|            null|            0.0|  23|
|15.04.2018 23:38|          null|          0.0|15.04.2018 23:38|            null|            0.0|  23|
|15.04.2018 23:40|          null|          0.0|15.04.2018 23:40|            null|            0.0|  23|
|20.04.2018 23:31|          null|          0.0|20.04.2018 23:31|            null|            0.0|  23|
+----------------+--------------+-------------+----------------+----------------+---------------+----+
only showing top 5 rows



In [94]:
third_table_final.filter((third_table_final.actual_arrival.isNull()) \
                   & (third_table_final.arrival_time.isNull()))\
                   .select('arrival_time', 
                           'actual_arrival', 
                           'arrival_delay', 
                           'departure_time', 
                           'actual_departure', 
                           'departure_delay',
                           'hour').show(5)

+------------+--------------+-------------+----------------+-------------------+---------------+----+
|arrival_time|actual_arrival|arrival_delay|  departure_time|   actual_departure|departure_delay|hour|
+------------+--------------+-------------+----------------+-------------------+---------------+----+
|        null|          null|         null|21.09.2017 05:41|21.09.2017 05:41:34|            0.0|   5|
|        null|          null|         null|27.09.2017 05:41|27.09.2017 05:41:26|            0.0|   5|
|        null|          null|         null|04.10.2017 05:41|04.10.2017 05:41:24|            0.0|   5|
|        null|          null|         null|10.10.2017 05:41|10.10.2017 05:41:24|            0.0|   5|
|        null|          null|         null|23.10.2017 05:41|23.10.2017 05:41:44|            0.0|   5|
+------------+--------------+-------------+----------------+-------------------+---------------+----+
only showing top 5 rows



In [102]:
third_table_final.filter((third_table_final.arrival_delay.isNull()) \
                   )\
                   .groupBy('AN_PROGNOSE_STATUS').count().show()

+------------------+-------+
|AN_PROGNOSE_STATUS|  count|
+------------------+-------+
|          PROGNOSE|2565026|
+------------------+-------+



- we count how many distinct bins of 1 minute delays we have:

In [97]:
third_table_final.groupBy('arrival_delay').count().orderBy('arrival_delay').count()

536

In [98]:
third_table_final.groupBy('arrival_delay').count().orderBy('arrival_delay').show(536)

+-------------+--------+
|arrival_delay|   count|
+-------------+--------+
|         null| 2565026|
|       -130.0|       1|
|        -89.0|       1|
|        -87.0|       1|
|        -86.0|       3|
|        -85.0|       1|
|        -84.0|       1|
|        -77.0|       3|
|        -76.0|      21|
|        -75.0|       1|
|        -74.0|       2|
|        -73.0|       2|
|        -72.0|       4|
|        -71.0|       4|
|        -70.0|       7|
|        -69.0|       8|
|        -68.0|       8|
|        -67.0|       8|
|        -66.0|       8|
|        -65.0|       7|
|        -64.0|       3|
|        -63.0|       6|
|        -62.0|       3|
|        -61.0|       6|
|        -60.0|       8|
|        -59.0|      16|
|        -58.0|      30|
|        -57.0|      50|
|        -56.0|      37|
|        -55.0|      31|
|        -54.0|      23|
|        -53.0|      25|
|        -52.0|      41|
|        -51.0|      23|
|        -50.0|      30|
|        -49.0|      34|
|        -48.0|      31|


In [99]:
third_table_final.groupBy('departure_delay').count().orderBy('departure_delay').show(536)

+---------------+--------+
|departure_delay|   count|
+---------------+--------+
|           null| 2566167|
|        -1425.0|       1|
|         -837.0|       1|
|         -130.0|       1|
|         -116.0|       1|
|         -107.0|       1|
|          -88.0|       2|
|          -87.0|       2|
|          -86.0|       1|
|          -85.0|       1|
|          -84.0|       2|
|          -77.0|       3|
|          -76.0|      23|
|          -75.0|       2|
|          -74.0|       1|
|          -73.0|       2|
|          -72.0|       3|
|          -71.0|       5|
|          -70.0|       7|
|          -69.0|       9|
|          -68.0|      11|
|          -67.0|       7|
|          -66.0|       8|
|          -65.0|       8|
|          -64.0|       5|
|          -63.0|       6|
|          -62.0|       3|
|          -61.0|       4|
|          -60.0|      14|
|          -59.0|      11|
|          -58.0|      31|
|          -57.0|      46|
|          -56.0|      40|
|          -55.0|      36|
|

- computing the mean of our delays:

In [100]:
third_table_final.select('arrival_delay').agg({"arrival_delay": "avg"}).show()

+------------------+
|avg(arrival_delay)|
+------------------+
|0.7878559556891835|
+------------------+



In [101]:
third_table_final.select('departure_delay').agg({"departure_delay": "avg"}).show()

+--------------------+
|avg(departure_delay)|
+--------------------+
|  0.8005839062540449|
+--------------------+



### STATISTICS TETS

- In this section we will produce statistics tests in order to see if our bins delays can fit well a known distribution (we were thinking about a logNormal distr).
- First we will produce the test on the whole dataset, that means on the third table.
- then we will run a test on the delays on each station (group by station name)
- after this we will do the same for each transport line (group by the line_id, right now the columns is erroneusly called 'train_line')
- then, the same thing for each trip (group by trip_id)
- finally, same tests for hour of the day (we need to choose how to split the day, right now the column 'hour' has 24 distinct values of course).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

Shall we consider that all delay distributions are log normals, no matter the groupby ? No function to guess the distrib, only tests to assess normality.

returns a tupple with two arrays, ticks and counts (pass number of bins into parameters, 20 here):

In [ ]:
#temp=third_table_final.select('departure_delay').rdd.flatMap(lambda x: x).histogram(20)

In [ ]:
#delay_per_station=third_table_final.groupBy('station_name','departure_delay').count().orderBy('station_name').collect()

In [ ]:
#def plot_hist_delay(bins,arrival_or_departure):
    #temp=third_table_final.select(arrival_or_departure).rdd.flatMap(lambda x: x).histogram(bins)
   # plt.bar(temp)

In [ ]:
#count_dep_delay_panda=third_table_final.groupBy('departure_delay').count().orderBy('departure_delay').toPandas()
#count_arr_delay_panda=third_table_final.groupBy('arrival_delay').count().orderBy('arrival_delay').toPandas()

We want to fit the lognormfit to all the delays per trip_id and line:

In [ ]:
trip_window = Window.partitionBy('trip_id')
train_line_window = Window.partitionBy('train_line')

In [ ]:
x=lognorm.fit(third_table_final['departure_delay'].over(trip_window).alias('fit'))

In [ ]:
 departure_delay_fit = third_table_final.select('trip_id','derparture_delay',x)

In [ ]:
sc.stop()